In [1]:
import tensorflow as tf
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# Datadirectory_train='/content/drive/MyDrive/ML_Project/train'
# Datadirectory_test='/content/drive/MyDrive/ML_Project/test'
# Classes=['sad','happy']

In [2]:
datasetdir = '/content/drive/MyDrive/ML_Project/train'
import os
os.chdir(datasetdir)
import matplotlib.pyplot as plt
import matplotlib.image as img
import tensorflow.keras as keras
import numpy as np

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
batch_size = 30
def generators(shape, preprocessing): 
    imgdatagen = ImageDataGenerator(
        preprocessing_function = preprocessing,
        horizontal_flip = True, 
        validation_split = 0.1,
    )

    height, width = shape

    train_dataset = imgdatagen.flow_from_directory(
        os.getcwd(),
        target_size = (height, width), 
        classes = ('sad','happy'),
        batch_size = batch_size,
        subset = 'training', 
    )

    val_dataset = imgdatagen.flow_from_directory(
        os.getcwd(),
        target_size = (height, width), 
        classes = ('sad','happy'),
        batch_size = batch_size,
        subset = 'validation'
    )
    return train_dataset, val_dataset

In [ ]:
vgg16 = keras.applications.vgg16
conv_model = vgg16.VGG16(weights='imagenet', include_top=False)
conv_model.summary()

In [5]:
train_dataset, val_dataset = generators((224,224), preprocessing=vgg16.preprocess_input)
conv_model = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))

Found 3600 images belonging to 2 classes.
Found 400 images belonging to 2 classes.


In [ ]:
x = keras.layers.Flatten()(conv_model.output)
x = keras.layers.Dense(100, activation='relu')(x)
x = keras.layers.Dense(100, activation='relu')(x)
x = keras.layers.Dense(100, activation='relu')(x)
predictions = keras.layers.Dense(2, activation='softmax')(x)
full_model = keras.models.Model(inputs=conv_model.input, outputs=predictions)
full_model.summary()

In [7]:
for layer in conv_model.layers:
    layer.trainable = False

In [ ]:
full_model.summary()

In [ ]:
full_model.compile(loss='binary_crossentropy',
                  optimizer=keras.optimizers.Adamax(lr=0.001),
                  metrics=['acc'])
history = full_model.fit_generator(
    train_dataset, 
    validation_data = val_dataset,
        workers=10,
    epochs=15,
)

In [ ]:
import pickle
with open('last_model', 'wb') as files:
    pickle.dump(full_model, files)